In [ ]:
!pip install datasets transformers[torch] accelerate -U

In [2]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1. Učitavanje podataka
print("1. Učitavanje podataka")
dataset = load_dataset("conll2003", trust_remote_code=True)

# 2. Priprema tokenizatora
print("\n2. Priprema tokenizatora")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# 3. Priprema oznaka
label_list = dataset["train"].features["ner_tags"].feature.names
label_encoding_dict = {i: label for i, label in enumerate(label_list)}

# 4. Funkcija za tokenizaciju
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 5. Tokenizacija dataset-a
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# 6. Priprema modela
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list))

# 7. Funkcija za računanje metrika
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Ignorišemo -100 oznake
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 8. Funkcija za treniranje
def train_model(model, tokenized_datasets, training_args):
    data_collator = DataCollatorForTokenClassification(tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    return trainer

# 9. Eksperimentisanje sa hiperparametrima
learning_rates = [1e-5, 3e-5, 5e-5]
batch_sizes = [16, 32]
epochs = [3, 5]

best_f1 = 0
best_model = None
best_params = None

for lr in learning_rates:
    for bs in batch_sizes:
        for ep in epochs:
            print(f"\nTreniranje sa lr={lr}, batch_size={bs}, epochs={ep}")

            training_args = TrainingArguments(
                output_dir=f"./results_lr{lr}_bs{bs}_ep{ep}",
                num_train_epochs=ep,
                per_device_train_batch_size=bs,
                per_device_eval_batch_size=bs,
                learning_rate=lr,
                weight_decay=0.01,
                evaluation_strategy="epoch",
                save_strategy="epoch",
                load_best_model_at_end=True,
                metric_for_best_model="f1"
            )

            model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list))
            trainer = train_model(model, tokenized_datasets, training_args)

            eval_results = trainer.evaluate()
            print(f"Evaluation results: {eval_results}")

            if eval_results["eval_f1"] > best_f1:
                best_f1 = eval_results["eval_f1"]
                best_model = model
                best_params = {"lr": lr, "batch_size": bs, "epochs": ep}

print(f"\nNajbolji model: F1 score = {best_f1}")
print(f"Najbolji parametri: {best_params}")

# 10. Čuvanje najboljeg modela
#best_model.save_pretrained("./best_ner_model")
#tokenizer.save_pretrained("./best_ner_model")

print("\nEksperiment završen!")

1. Učitavanje podataka


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.049621,0.985145,0.985002,0.985219,0.985145
2,0.117200,0.047344,0.987403,0.987308,0.987294,0.987403
3,0.026800,0.048019,0.987695,0.987616,0.987563,0.987695


Evaluation results: {'eval_loss': 0.04801937937736511, 'eval_accuracy': 0.9876951832093765, 'eval_f1': 0.9876155106846302, 'eval_precision': 0.9875626718491455, 'eval_recall': 0.9876951832093765, 'eval_runtime': 4.9043, 'eval_samples_per_second': 662.679, 'eval_steps_per_second': 20.798, 'epoch': 3.0}

Treniranje sa lr=5e-05, batch_size=32, epochs=5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.051374,0.984405,0.984291,0.984657,0.984405
2,0.114600,0.045423,0.988124,0.988043,0.988032,0.988124
3,0.026800,0.047470,0.987715,0.987562,0.987504,0.987715
4,0.011900,0.052078,0.988085,0.988005,0.987987,0.988085
5,0.006800,0.052267,0.988357,0.988286,0.988246,0.988357


Evaluation results: {'eval_loss': 0.052266668528318405, 'eval_accuracy': 0.9883571512012772, 'eval_f1': 0.9882856765854345, 'eval_precision': 0.988246201386865, 'eval_recall': 0.9883571512012772, 'eval_runtime': 4.7571, 'eval_samples_per_second': 683.187, 'eval_steps_per_second': 21.442, 'epoch': 5.0}

Najbolji model: F1 score = 0.9883537364482007
Najbolji parametri: {'lr': 5e-05, 'batch_size': 16, 'epochs': 3}

Eksperiment završen!
